<a href="https://colab.research.google.com/github/KSG2001/AI_Study/blob/UEFA/UEFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UEFA 챔피언스 리그

In [ ]:
import sys
print(sys.version)

3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]


In [ ]:
!conda install python-graphviz

/bin/bash: conda: command not found


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import cv2
import graphviz

In [ ]:
graphviz.__version__

'0.10.1'

In [ ]:
from google.colab import drive

!jupyter nbconvert --to markdown "/content/gdrive/MyDrive/lab_study/UEFA.ipynb"

drive.mount('/content/gdrive')
print('/content/gdrive/lab_study/UEFA.txt')
f= open("/content/gdrive/MyDrive/lab_study/UEFA.txt",'r')
data=f.read().splitlines()
UEFA=[x.split() for x in data]
print(UEFA)

[NbConvertApp] WARNING | pattern '/content/gdrive/MyDrive/lab_study/UEFA.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.ans

## 함수 define 부분

### 포트를 조로 변경해주는 함수

In [ ]:
def Team_selet(port1,port2,port3,port4):
  team=[]
  for i in range(len(port1)):
    team=team+[port1[i]]
    team=team+[port2[i]]
    team=team+[port3[i]]
    team=team+[port4[i]]
  groupA=team[0:4]
  groupB=team[4:8]
  groupC=team[8:12]
  groupD=team[12:16]
  groupE=team[16:20]
  groupF=team[20:24]
  groupG=team[24:28]
  groupH=team[28:32]
  return groupA,groupB,groupC,groupD,groupE,groupF,groupG,groupH  

### 팀별 점수 부여 함수

In [ ]:
def Score(home,go):

  # 각 팀마다 랜덤 점수 부여
  home_goal = random.choices(range(0,6), weights= [0.2,0.25,0.25,0.15,0.1,0.05]) # 홈에서 경기하는 팀
  go_goal = random.choices(range(0,6), weights= [0.2,0.25,0.25,0.15,0.1,0.05]) # 원정 가서 경기하는 팀

  home_score = go_score = 0

  # 규칙1 승점 차이 
  if home_goal > go_goal:
    home_score += 3 * 1000
    go_score += 0 * 1000
  elif home_goal == go_goal:
    home_score += 1 * 1000
    go_score += 1 * 1000
  else:
    home_score += 0 * 1000
    go_score += 3 * 1000

  # 규칙2 득실차
  if home_goal > go_goal:
    home_score += ((home_goal[0] - go_goal[0]) * 100)
    go_score += ((home_goal[0] - go_goal[0]) * 100)

  elif home_goal == go_goal:
    home_score += ((home_goal[0] - go_goal[0]) * 100)
    go_score += ((home_goal[0] - go_goal[0]) * 100)
  else:
    home_score += ((go_goal[0] - home_goal[0]) * 100)
    go_score += ((go_goal[0] - home_goal[0]) * 100)

  # 규칙3 다득점
  home_score += home_goal[0] * 10
  go_score += go_goal[0] * 10

  # 규칙4 원정 & 홈
  home_score += 0
  go_score += 5

  return home_score, go_score

### 승부차기 점수 부여 함수

In [ ]:
def Score_shoot(A,B):

  # A의 홈경기
  score_Ah,score_Bg=Score(A,B)

  # B의 홈경기
  score_Bh,score_Ag=Score(B,A)

  score_At = score_Ah + score_Ag
  score_Bt = score_Bh + score_Bg

  if score_At==score_Bt:
    sc_sh_A = random.randrange(0,6)
    sc_sh_B = random.randrange(0,6)

    score_At += sc_sh_A
    score_Bt += sc_sh_B

    while score_At==score_Bt:
      sc_sh_A_1 = random.randrange(0,2)
      sc_sh_B_1 = random.randrange(0,2)

      score_At += sc_sh_A_1
      score_Bt += sc_sh_B_1


  return score_At, score_Bt

### 32강 경기 함수

In [ ]:
def Ranking(A,B,C,D):

  rank = []

  # 1 ~ 12 번째 까지의 경기
  # Home 경기
  score_A_1,score_B_1=Score(A,B)
  score_A_2,score_C_2=Score(A,C)
  score_A_3,score_D_1=Score(A,D)
  score_B_2,score_C_3=Score(B,C)
  score_B_3,score_D_2=Score(B,D)
  score_C_1,score_D_3=Score(C,D)

  # go 경기
  score_B_4,score_A_4=Score(B,A)
  score_C_4,score_A_5=Score(C,A)
  score_D_4,score_A_6=Score(D,A)
  score_C_5,score_B_5=Score(C,B)
  score_D_5,score_B_6=Score(D,B)
  score_D_6,score_C_6=Score(D,C)

  # 각 팀마다의 경기 점수 총합
  team_A = score_A_1 + score_A_2 + score_A_3 + score_A_4 + score_A_5 + score_A_6 
  team_B = score_B_1 + score_B_2 + score_B_3 + score_B_4 + score_B_5 + score_B_6 
  team_C = score_C_1 + score_C_2 + score_C_3 + score_C_4 + score_C_5 + score_C_6 
  team_D = score_D_1 + score_D_2 + score_D_3 + score_D_4 + score_D_5 + score_D_6 

  rank.append([team_A,A])
  rank.append([team_B,B])
  rank.append([team_C,C])
  rank.append([team_D,D])

  rank.sort(reverse=True)

  return rank[0][1], rank[1][1], team_A, team_B, team_C, team_D

### 토너먼트 경기 함수

In [ ]:
def Match(A,B):

  Match = []

  score_A, score_B = Score_shoot(A,B)

  Match.append([score_A,A])
  Match.append([score_B,B])

  Match.sort(reverse=True)

  return Match[0][1],  Match[0][0], Match[1][0]

### 시각화에 필요한 함수

In [ ]:
def save_graph_as_svg(dot_string, output_file_name):
    if type(dot_string) is str:
        g = graphviz.Source(dot_string)
    elif isinstance(dot_string, (graphviz.dot.Digraph, graphviz.dot.Graph)):
        g = dot_string
    g.format='svg'
    g.filename = output_file_name
    g.directory = '../../assets/images/markdown_img/'
    g.render(view=False)
    return g

## UEFA 경기 

### 포트 & 그룹 편성

#### 포트 편성

In [ ]:
UEFA_name = sum(UEFA,[])
UEFA_name_port = sum(UEFA,[])

port1 = random.sample(UEFA_name_port, 8)
for i in range(len(port1)):
  UEFA_name_port.remove(port1[i])


port2 = random.sample(UEFA_name_port, 8)
for i in range(len(port2)):
  UEFA_name_port.remove(port2[i])

port3 = random.sample(UEFA_name_port, 8)
for i in range(len(port3)):
  UEFA_name_port.remove(port3[i])

port4 = UEFA_name_port

#### 그룹 편성

In [ ]:
a=Team_selet(port1,port2,port3,port4)
groupA=a[0]
groupB=a[1]
groupC=a[2]
groupD=a[3]
groupE=a[4]
groupF=a[5]
groupG=a[6]
groupH=a[7]

print(groupA)
print(groupB) 
print(groupC) 
print(groupD)
print(groupE)
print(groupF)
print(groupG)
print(groupH)

In [ ]:
#나라 중복 검사
AAA=[]
BBB=[]
CCC=[]
DDD=[]
EEE=[]
FFF=[]
GGG=[]
HHH=[]
for i in range(4):
  AA=groupA[i]
  AAA=AAA+[AA[0:2]]
  BB=groupB[i]
  BBB=BBB+[BB[0:2]]
  CC=groupC[i]
  CCC=CCC+[CC[0:2]]
  DD=groupD[i]
  DDD=DDD+[DD[0:2]]  
  DD=groupD[i]
  DDD=DDD+[DD[0:2]]  
  EE=groupE[i]
  EEE=EEE+[EE[0:2]]  
  FF=groupF[i]
  FFF=FFF+[FF[0:2]]  
  GG=groupG[i]
  GGG=GGG+[GG[0:2]]    
  HH=groupH[i]
  HHH=HHH+[HH[0:2]]   

result_A=list(np.unique(AAA))
result_B=list(np.unique(BBB))
result_C=list(np.unique(CCC))
result_D=list(np.unique(DDD))
result_E=list(np.unique(EEE))
result_F=list(np.unique(FFF))
result_G=list(np.unique(GGG))
result_H=list(np.unique(HHH))

#중복일 경우, 다시 shuffle로 다시 group 배정!
while len(result_A)!=4 or len(result_B)!=4 or len(result_C)!=4 or len(result_D)!=4 or len(result_E)!=4 or len(result_F)!=4 or  len(result_G)!=4 or len(result_H)!=4:
  random.shuffle(port1)
  random.shuffle(port2)
  random.shuffle(port3)
  random.shuffle(port4)  
  a=Team_selet(port1,port2,port3,port4)
  groupA=a[0]
  groupB=a[1]
  groupC=a[2]
  groupD=a[3]
  groupE=a[4]
  groupF=a[5]
  groupG=a[6]
  groupH=a[7]
  AAA=[]
  BBB=[]
  CCC=[]
  DDD=[]
  EEE=[]
  FFF=[]
  GGG=[]
  HHH=[]
  for i in range(4):
    AA=groupA[i]
    AAA=AAA+[AA[0:2]]
    BB=groupB[i]
    BBB=BBB+[BB[0:2]]
    CC=groupC[i]
    CCC=CCC+[CC[0:2]]
    DD=groupD[i]
    DDD=DDD+[DD[0:2]]  
    DD=groupD[i]
    DDD=DDD+[DD[0:2]]  
    EE=groupE[i]
    EEE=EEE+[EE[0:2]]  
    FF=groupF[i]
    FFF=FFF+[FF[0:2]]  
    GG=groupG[i]
    GGG=GGG+[GG[0:2]]    
    HH=groupH[i]
    HHH=HHH+[HH[0:2]]   

  result_A=list(np.unique(AAA))
  result_B=list(np.unique(BBB))
  result_C=list(np.unique(CCC))
  result_D=list(np.unique(DDD))
  result_E=list(np.unique(EEE))
  result_F=list(np.unique(FFF))
  result_G=list(np.unique(GGG))
  result_H=list(np.unique(HHH))

#최종 group
print(groupA)
print(groupB) 
print(groupC) 
print(groupD)
print(groupE)
print(groupF)
print(groupG)
print(groupH)

### 32강

#### 32강 경기

In [ ]:
groupA_1, groupA_2, grA_1_sc, grA_2_sc, grA_3_sc, grA_4_sc = Ranking(groupA[0],groupA[1],groupA[2],groupA[3]) # 팀 A의 1,2위

groupB_1, groupB_2, grB_1_sc, grB_2_sc, grB_3_sc, grB_4_sc = Ranking(groupB[0],groupB[1],groupB[2],groupB[3]) # 팀 B의 1,2위

groupC_1, groupC_2, grC_1_sc, grC_2_sc, grC_3_sc, grC_4_sc = Ranking(groupC[0],groupC[1],groupC[2],groupC[3]) # 팀 C의 1,2위

groupD_1, groupD_2, grD_1_sc, grD_2_sc, grD_3_sc, grD_4_sc = Ranking(groupD[0],groupD[1],groupD[2],groupD[3]) # 팀 D의 1,2위

groupE_1, groupE_2, grE_1_sc, grE_2_sc, grE_3_sc, grE_4_sc = Ranking(groupE[0],groupE[1],groupE[2],groupE[3]) # 팀 E의 1,2위

groupF_1, groupF_2, grF_1_sc, grF_2_sc, grF_3_sc, grF_4_sc = Ranking(groupF[0],groupF[1],groupF[2],groupF[3]) # 팀 F의 1,2위

groupG_1, groupG_2, grG_1_sc, grG_2_sc, grG_3_sc, grG_4_sc = Ranking(groupG[0],groupG[1],groupG[2],groupG[3]) # 팀 G의 1,2위

groupH_1, groupH_2, grH_1_sc, grH_2_sc, grH_3_sc, grH_4_sc = Ranking(groupH[0],groupH[1],groupH[2],groupH[3]) # 팀 H의 1,2위



print(groupA_1, groupA_2,groupB_1, groupB_2,groupC_1, groupC_2,groupD_1, groupD_2,groupE_1, groupE_2,groupF_1, groupF_2,groupG_1, groupG_2,groupH_1, groupH_2)

#### 32강 경기 시각화

In [ ]:
digraph_32 = graphviz.Graph(comment="The Round Table")
digraph_32.node("32강")
digraph_32.node("groupA")
digraph_32.node("groupB")
digraph_32.node("groupC")
digraph_32.node("groupD")
digraph_32.node("groupE")
digraph_32.node("groupF")
digraph_32.node("groupG")
digraph_32.node("groupH")
digraph_32.edge("32강", "groupA")
digraph_32.edge("32강", "groupB")
digraph_32.edge("32강", "groupC")
digraph_32.edge("32강", "groupD")
digraph_32.edge("32강", "groupE")
digraph_32.edge("32강", "groupF")
digraph_32.edge("32강", "groupG")
digraph_32.edge("32강", "groupH")

for i in range(0,4):
  digraph_32.node(groupA[i])
  digraph_32.edge("groupA", groupA[i])
  digraph_32.node(groupB[i])
  digraph_32.edge("groupB", groupB[i])  
  digraph_32.node(groupC[i])
  digraph_32.edge("groupC", groupC[i])  
  digraph_32.node(groupD[i])
  digraph_32.edge("groupD", groupD[i])  
  digraph_32.node(groupE[i])
  digraph_32.edge("groupE", groupE[i])    
  digraph_32.node(groupF[i])
  digraph_32.edge("groupF", groupF[i])  
  digraph_32.node(groupG[i])
  digraph_32.edge("groupG", groupG[i])  
  digraph_32.node(groupH[i])
  digraph_32.edge("groupH", groupH[i])    

save_graph_as_svg(digraph_32, 'round table')

### 16강

#### 16강 경기

In [ ]:
match_16_1, win_16_1, lose_16_1 = Match(groupA_1,groupB_2) # 팀A 1위, 팀B 2위 경기 (16강 1조)

match_16_2, win_16_2, lose_16_2 = Match(groupB_1,groupA_2) # 팀B 1위, 팀A 2위 경기 (16강 2조)

match_16_3, win_16_3, lose_16_3 = Match(groupC_1,groupD_2) # 팀C 1위, 팀D 2위 경기 (16강 3조)

match_16_4, win_16_4, lose_16_4 = Match(groupD_1,groupC_2) # 팀D 1위, 팀C 2위 경기 (16강 4조)

match_16_5, win_16_5, lose_16_5 = Match(groupE_1,groupF_2) # 팀E 1위, 팀F 2위 경기 (16강 5조)

match_16_6, win_16_6, lose_16_6 = Match(groupF_1,groupE_2) # 팀F 1위, 팀E 2위 경기 (16강 6조)

match_16_7, win_16_7, lose_16_7 = Match(groupG_1,groupH_2) # 팀G 1위, 팀H 2위 경기 (16강 7조)

match_16_8, win_16_8, lose_16_8 = Match(groupH_1,groupG_2) # 팀H 1위, 팀G 2위 경기 (16강 8조)



print(match_16_1,match_16_2,match_16_3,match_16_4,match_16_5,match_16_6,match_16_7,match_16_8)

#### 16강 경기 시각화

In [ ]:
digraph_16 = graphviz.Graph(comment="The Round Table")

digraph_16.edge("16강", match_16_1 + "_16")
digraph_16.edge("16강", match_16_2 + "_16")
digraph_16.edge("16강", match_16_3 + "_16")
digraph_16.edge("16강", match_16_4 + "_16")
digraph_16.edge("16강", match_16_5 + "_16")
digraph_16.edge("16강", match_16_6 + "_16")
digraph_16.edge("16강", match_16_7 + "_16")
digraph_16.edge("16강", match_16_8 + "_16")

digraph_16.edge(match_16_1 + "_16",groupA_1)
digraph_16.edge(match_16_1 + "_16",groupB_2)
digraph_16.edge(match_16_2 + "_16",groupA_2)
digraph_16.edge(match_16_2 + "_16",groupB_1)
digraph_16.edge(match_16_3 + "_16",groupC_1)
digraph_16.edge(match_16_3 + "_16",groupD_2)
digraph_16.edge(match_16_4 + "_16",groupC_2)
digraph_16.edge(match_16_4 + "_16",groupD_1)
digraph_16.edge(match_16_5 + "_16",groupE_1)
digraph_16.edge(match_16_5 + "_16",groupF_2)
digraph_16.edge(match_16_6 + "_16",groupE_2)
digraph_16.edge(match_16_6 + "_16",groupF_1)
digraph_16.edge(match_16_7 + "_16",groupG_1)
digraph_16.edge(match_16_7 + "_16",groupH_2)
digraph_16.edge(match_16_8 + "_16",groupG_2)
digraph_16.edge(match_16_8 + "_16",groupH_1)


save_graph_as_svg(digraph_16, 'round table')

### 8강

#### 8강 경기

In [ ]:
match_8_1, win_8_1, lose_8_1 = Match(match_16_1,match_16_2) # 16강 1조, 2조 경기 (8강 1조)

match_8_2, win_8_2, lose_8_2 = Match(match_16_3,match_16_4) # 16강 3조, 4조 경기 (8강 2조)

match_8_3, win_8_3, lose_8_3 = Match(match_16_5,match_16_6) # 16강 5조, 6조 경기 (8강 3조)

match_8_4, win_8_4, lose_8_4 = Match(match_16_7,match_16_8) # 16강 7조, 8조 경기 (8강 4조)



print(match_8_1,match_8_2,match_8_3,match_8_4)

#### 8강 경기 시각화

In [ ]:
digraph_8 = graphviz.Graph(comment="The Round Table")
digraph_8.node("8강")
digraph_8.node(match_16_1)
digraph_8.node(match_16_2)
digraph_8.node(match_16_3)
digraph_8.node(match_16_4)
digraph_8.node(match_16_5)
digraph_8.node(match_16_6)
digraph_8.node(match_16_7)
digraph_8.node(match_16_8)
digraph_8.node(match_8_1)
digraph_8.node(match_8_2)
digraph_8.node(match_8_3)
digraph_8.node(match_8_4)

digraph_8.edge("8강", match_8_1 + "_8")
digraph_8.edge("8강", match_8_2 + "_8")
digraph_8.edge("8강", match_8_3 + "_8")
digraph_8.edge("8강", match_8_4 + "_8")

digraph_8.edge(match_8_1+ "_8", match_16_1)
digraph_8.edge(match_8_1+ "_8", match_16_2)
digraph_8.edge(match_8_2+ "_8", match_16_3)
digraph_8.edge(match_8_2+ "_8", match_16_4)
digraph_8.edge(match_8_3+ "_8", match_16_5)
digraph_8.edge(match_8_3+ "_8", match_16_6)
digraph_8.edge(match_8_4+ "_8", match_16_7)
digraph_8.edge(match_8_4+ "_8", match_16_8)


save_graph_as_svg(digraph_8, 'round table')

### 4강

#### 4강 경기

In [ ]:
match_4_1, win_4_1, lose_4_1 = Match(match_8_1,match_8_2) # 8강 1,2조

match_4_2, win_4_2, lose_4_2 = Match(match_8_3,match_8_4) # 8강 3,4조



print(match_4_1,match_4_2)

#### 4강 경기 시각화

In [ ]:
digraph_4 = graphviz.Graph(comment="The Round Table")
digraph_4.node("4강")
digraph_4.node(match_8_1)
digraph_4.node(match_8_2)
digraph_4.node(match_8_3)
digraph_4.node(match_8_4)
digraph_4.node(match_4_1)
digraph_4.node(match_4_2)

digraph_4.edge("4강", match_4_1 + "_4")
digraph_4.edge("4강", match_4_2 + "_4")

digraph_4.edge(match_4_1+ "_4", match_8_1)
digraph_4.edge(match_4_1+ "_4", match_8_2)
digraph_4.edge(match_4_2+ "_4", match_8_3)
digraph_4.edge(match_4_2+ "_4", match_8_4)


save_graph_as_svg(digraph_4, 'round table')

### 결승전

#### 결승전 경기

In [ ]:
winer, win_2_1, lose_2_1 = Match(match_4_1,match_4_2) # 결승




print(winer)

#### 결승전 경기 시각화

In [ ]:
digraph_2 = graphviz.Graph(comment="The Round Table")
digraph_2.node("결승전")
digraph_2.node(match_4_1)
digraph_2.node(match_4_2)
digraph_2.node(winer)

digraph_2.edge("결승전", winer + "_winer")


digraph_2.edge(winer+ "_winer", match_4_1)
digraph_2.edge(winer+ "_winer", match_4_2)


save_graph_as_svg(digraph_2, 'round table')

### 전체 경기 시각화 

#### 16강 ~ 결승전

In [ ]:
digraph_16T = graphviz.Graph(comment="The Round Table")

digraph_16T.edge(winer+ "_winer", match_4_1+ "_4", "  결승전")
digraph_16T.edge(winer+ "_winer", match_4_2+ "_4")

digraph_16T.edge(match_4_1+ "_4", match_8_1+ "_8", "  4강")
digraph_16T.edge(match_4_1+ "_4", match_8_2+ "_8")
digraph_16T.edge(match_4_2+ "_4", match_8_3+ "_8")
digraph_16T.edge(match_4_2+ "_4", match_8_4+ "_8")

digraph_16T.edge(match_8_1+ "_8", match_16_1+ "_16", "  8강")
digraph_16T.edge(match_8_1+ "_8", match_16_2+ "_16")
digraph_16T.edge(match_8_2+ "_8", match_16_3+ "_16")
digraph_16T.edge(match_8_2+ "_8", match_16_4+ "_16")
digraph_16T.edge(match_8_3+ "_8", match_16_5+ "_16")
digraph_16T.edge(match_8_3+ "_8", match_16_6+ "_16")
digraph_16T.edge(match_8_4+ "_8", match_16_7+ "_16")
digraph_16T.edge(match_8_4+ "_8", match_16_8+ "_16")

digraph_16T.edge(match_16_1 + "_16",groupA_1 + "_GrA_1", "  16강")
digraph_16T.edge(match_16_1 + "_16",groupB_2 + "_GrB_2")
digraph_16T.edge(match_16_2 + "_16",groupA_2 + "_GrA_2")
digraph_16T.edge(match_16_2 + "_16",groupB_1 + "_GrB_1")
digraph_16T.edge(match_16_3 + "_16",groupC_1 + "_GrC_1")
digraph_16T.edge(match_16_3 + "_16",groupD_2 + "_GrD_2")
digraph_16T.edge(match_16_4 + "_16",groupC_2 + "_GrC_2")
digraph_16T.edge(match_16_4 + "_16",groupD_1 + "_GrD_1")
digraph_16T.edge(match_16_5 + "_16",groupE_1 + "_GrE_1")
digraph_16T.edge(match_16_5 + "_16",groupF_2 + "_GrF_2")
digraph_16T.edge(match_16_6 + "_16",groupE_2 + "_GrE_2")
digraph_16T.edge(match_16_6 + "_16",groupF_1 + "_GrF_1")
digraph_16T.edge(match_16_7 + "_16",groupG_1 + "_GrG_1")
digraph_16T.edge(match_16_7 + "_16",groupH_2 + "_GrH_2")
digraph_16T.edge(match_16_8 + "_16",groupG_2 + "_GrG_2")
digraph_16T.edge(match_16_8 + "_16",groupH_1 + "_GrH_1")

save_graph_as_svg(digraph_16T, 'round table')

#### 32강 ~ 결승전

In [ ]:
digraph_T = graphviz.Graph(comment="The Round Table")

digraph_T.edge(winer+ "_winer", match_4_1+ "_4", "  결승전")
digraph_T.edge(winer+ "_winer", match_4_2+ "_4")

digraph_T.edge(match_4_1+ "_4", match_8_1+ "_8", "  4강")
digraph_T.edge(match_4_1+ "_4", match_8_2+ "_8")
digraph_T.edge(match_4_2+ "_4", match_8_3+ "_8")
digraph_T.edge(match_4_2+ "_4", match_8_4+ "_8")


digraph_T.edge(match_8_1+ "_8", match_16_1+ "_16", "  8강")
digraph_T.edge(match_8_1+ "_8", match_16_2+ "_16")
digraph_T.edge(match_8_2+ "_8", match_16_3+ "_16")
digraph_T.edge(match_8_2+ "_8", match_16_4+ "_16")
digraph_T.edge(match_8_3+ "_8", match_16_5+ "_16")
digraph_T.edge(match_8_3+ "_8", match_16_6+ "_16")
digraph_T.edge(match_8_4+ "_8", match_16_7+ "_16")
digraph_T.edge(match_8_4+ "_8", match_16_8+ "_16")

digraph_T.edge(match_16_1 + "_16",groupA_1 + "_GrA_1", "  16강")
digraph_T.edge(match_16_1 + "_16",groupB_2 + "_GrB_2")
digraph_T.edge(match_16_2 + "_16",groupA_2 + "_GrA_2")
digraph_T.edge(match_16_2 + "_16",groupB_1 + "_GrB_1")
digraph_T.edge(match_16_3 + "_16",groupC_1 + "_GrC_1")
digraph_T.edge(match_16_3 + "_16",groupD_2 + "_GrD_2")
digraph_T.edge(match_16_4 + "_16",groupC_2 + "_GrC_2")
digraph_T.edge(match_16_4 + "_16",groupD_1 + "_GrD_1")
digraph_T.edge(match_16_5 + "_16",groupE_1 + "_GrE_1")
digraph_T.edge(match_16_5 + "_16",groupF_2 + "_GrF_2")
digraph_T.edge(match_16_6 + "_16",groupE_2 + "_GrE_2")
digraph_T.edge(match_16_6 + "_16",groupF_1 + "_GrF_1")
digraph_T.edge(match_16_7 + "_16",groupG_1 + "_GrG_1")
digraph_T.edge(match_16_7 + "_16",groupH_2 + "_GrH_2")
digraph_T.edge(match_16_8 + "_16",groupG_2 + "_GrG_2")
digraph_T.edge(match_16_8 + "_16",groupH_1 + "_GrH_1")

digraph_T.edge(groupA_1 + "_GrA_1", "groupA")
digraph_T.edge(groupA_2 + "_GrA_2", "groupA")
digraph_T.edge(groupB_1 + "_GrB_1", "groupB")
digraph_T.edge(groupB_2 + "_GrB_2", "groupB")
digraph_T.edge(groupC_1 + "_GrC_1", "groupC")
digraph_T.edge(groupD_1 + "_GrD_1", "groupD")
digraph_T.edge(groupE_1 + "_GrE_1", "groupE")
digraph_T.edge(groupF_1 + "_GrF_1", "groupF")
digraph_T.edge(groupG_1 + "_GrG_1", "groupG")
digraph_T.edge(groupH_1 + "_GrH_1", "groupH")
digraph_T.edge(groupC_2 + "_GrC_2", "groupC")
digraph_T.edge(groupD_2 + "_GrD_2", "groupD")
digraph_T.edge(groupE_2 + "_GrE_2", "groupE")
digraph_T.edge(groupF_2 + "_GrF_2", "groupF")
digraph_T.edge(groupG_2 + "_GrG_2", "groupG")
digraph_T.edge(groupH_2 + "_GrH_2", "groupH")

for i in range(0,4):
  digraph_T.node(groupA[i])
  digraph_T.edge("groupA", groupA[i])
  digraph_T.node(groupB[i])
  digraph_T.edge("groupB", groupB[i])  
  digraph_T.node(groupC[i])
  digraph_T.edge("groupC", groupC[i])  
  digraph_T.node(groupD[i])
  digraph_T.edge("groupD", groupD[i])  
  digraph_T.node(groupE[i])
  digraph_T.edge("groupE", groupE[i])    
  digraph_T.node(groupF[i])
  digraph_T.edge("groupF", groupF[i])  
  digraph_T.node(groupG[i])
  digraph_T.edge("groupG", groupG[i])  
  digraph_T.node(groupH[i])
  digraph_T.edge("groupH", groupH[i])  

save_graph_as_svg(digraph_T, 'round table')